In [89]:
import pandas as pd

In [2]:
df = pd.read_csv("data/gathered_data.csv")

In [3]:
df_cnn = df[list(map(lambda x: 'cnn.co.jp' in x, df['url'].tolist()))].dropna()
df_reuters = df[list(map(lambda x: 'reuters.com' in x, df['url'].tolist()))].dropna()
df_afp = df[list(map(lambda x: 'afpbb.com' in x, df['url'].tolist()))].dropna()

In [4]:
import re
import datetime
from dateutil import parser

def jadate_to_datetime(jadate, site_type):
    if site_type=="afp":
        t = map(int, re.sub(r"(年|月|日|:)"," ", jadate).split())
        return pd.Timestamp(datetime.datetime(*t, 0))
    elif site_type=="cnn":
        tmp = re.sub(r"([0-9\.]+).+at ([0-9:]+).+", r"\1,\2", jadate).split(",")
        t = list(map(int, tmp[0].split(".") + tmp[1].split(":")))
        return pd.Timestamp(datetime.datetime(*t, 0))
    elif site_type=="reuters":
        tmp = re.sub(r"(.+)/.+", r"\1", jadate).replace("/","").replace(",", "")
        tmp = re.sub(r"( +)", r" ", tmp)
        tmp = tmp.split()
        tmp2 = tmp[3].split(":")
        tmp2[0] = tmp2[0].zfill(2)
        tmp2[1] = tmp2[1].zfill(2)
        tmp[3] = ':'.join(tmp2)
        data = ' '.join(tmp)
        return pd.Timestamp(parser.parse(data))

In [8]:
from tqdm import tqdm_notebook
site_types = ["afp", "cnn", "reuters"]
dfs = [df_afp, df_cnn, df_reuters]
results_day = []
for site_type, df in zip(site_types, dfs):
    day_fixed = []
    for i, d in tqdm_notebook(df.iterrows()):
        day_fixed.append(jadate_to_datetime(d['day'], site_type))
    results_day.append(day_fixed)

In [9]:
df_afp['day_fixed'] = results_day[0]
df_cnn['day_fixed'] = results_day[1]
df_reuters['day_fixed'] = results_day[2]

In [10]:
df = pd.concat([df_afp, df_cnn, df_reuters])
df.to_csv("fixed_data.csv")
df.head()

,body,day,title,url,day_fixed
0,世界的なブランドを評定する世界品牌実験室（World Brand Lab）はこのほど、「中国...,2018年7月5日 10:00,青島ビール、ブランド価値は約2兆円,http://www.afpbb.com/articles/-/3181154,2018-07-05 10:00:00
1,国連児童基金（ユニセフ、UNICEF）は3日、イエメンでの3年にわたる内戦が子どもたちに深刻...,2018年7月4日 22:31,イエメン内戦、3年で子ども2200人死亡 ユニセフが「大虐殺」を非難,http://www.afpbb.com/articles/-/3181229,2018-07-04 22:31:00
2,国連のゼイド・ラアド・アル・フセイン（Zeid Ra'ad Al Hussein）人権高等弁...,2018年7月5日 5:20,ミャンマーがロヒンギャ帰還者58人拘束 国連報告,http://www.afpbb.com/articles/-/3181240,2018-07-05 05:20:00
3,インドでメッセージサービス「ワッツアップ（WhatsApp）」上に流布するデマを信じた人々に...,2018年7月4日 19:27,デマによるリンチ、インドで相次ぐ ワッツアップが対処を約束,http://www.afpbb.com/articles/-/3181220,2018-07-04 19:27:00
4,英警察は3日、北西部チェスター（Chester）の病院で起きた新生児不審死の捜査結果を受けて...,2018年7月4日 16:50,英病院で新生児の不審死8件、医療従事者の女を逮捕,http://www.afpbb.com/articles/-/3181191,2018-07-04 16:50:00


In [5]:
df = pd.read_csv("fixed_data.csv")

In [15]:
df['day_fixed'] = pd.to_datetime(df['day_fixed'])

In [16]:
df_afp = df[list(map(lambda x: 'afpbb.com' in x, df['url'].tolist()))]

In [8]:
import numpy as np
from functools import partial
from dateutil import parser

def diff_date(t1, t2):
    if t1 < t2:
        tmp = t1
        t1 = t2
        t2 = tmp
    return (t1-t2).to_timedelta64().astype("timedelta64[h]")/np.timedelta64(1,'h')

def within(t1, base, hour=12):
    return diff_date(t1, base) <= hour

def get_around(df, base, hour=12, column="day_fixed"):
    func = partial(within, base=base, hour=hour)
    return df[list(map(func, df[column].tolist()))]

def get_day(start, plus):
    return pd.Timestamp(start + datetime.timedelta(days=plus))

In [17]:
import MeCab
def extract_data(d, tagger, column1, column2):
    return tagger.parse(d[1][column1] + " " + d[1][column2].split("。")[0])

tagger = MeCab.Tagger("-Owakati")
func = partial(extract_data, tagger=tagger, column1="title", column2="body")

df_afp['data'] = list(map(func, df_afp.iterrows()))
df['data'] = list(map(func, df.iterrows()))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
df.to_csv("progress_data.csv", index=False)

In [18]:
start = datetime.datetime(2017,2,1,12,0,0)
day = get_day(start, 0)
targets_afp = get_around(df_afp, day, hour=12*30)
targets = get_around(df, day, hour=12*30)

In [178]:
import scipy.sparse as sps
from tqdm import tqdm_notebook

def mul_sim(similarities, days):
    for i, d1 in enumerate(tqdm_notebook(days)):
        d = np.array([1/np.exp(diff_date(d1, d2)/24.0) for d2 in days])
        similarities[i, :] = similarities[i, :].multiply(d)
    return similarities

days =targets_afp['day_fixed'].tolist()

In [179]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import itertools
import markov_clustering as mc
stps = stopwords.words("japanese")
stps = stps + [''.join(list(stp)) for stp in list(itertools.product(stps, stps))]
count_vectors = CountVectorizer(stop_words=stps).fit_transform(targets_afp['data'])
similarities = cosine_similarity(count_vectors, dense_output=False)
similarities = mul_sim(similarities, days)
result = mc.run_mcl(similarities)
clusters = mc.get_clusters(result)

In [26]:
import pickle
with open("clustering.pkl","wb") as f:
    pickle.dump((result, clusters), f)

In [180]:
for i, cluster in enumerate(clusters):
    if len(cluster) > 1:
        print("[[[cluster {}]]]".format(i))
        for i, c in enumerate(cluster[:3]):
            print("{}) :{}".format(i, targets_afp.iloc[c]['data']))
        print("")

[[[cluster 8]]]
0) :トランプ 氏 の 選対 メンバー 、 ロシア 情報 当局 者 と 繰り返し 接触 米 紙 昨年 の 米 大統領 選 で ドナルド ・ トランプ （ Donald Trump ） 氏 の 選挙 対策 チーム の メンバー だっ た 複数 の 人物 が 、 たびたび ロシア の 情報 当局 の 幹部 と 接触 し て い た こと が 分かっ た と 、 14 日 付 の 米 紙 ニューヨーク・タイムズ （ New York Times ） が 報じ た 

1) :ロシア 、 巡航 ミサイル を 新 配備 か 米 「 軍縮 条約 に 違反 」 と 警告 米 紙 ニューヨーク・タイムズ （ New York Times ） は 14 日 、 ロシア が 新た に 地上 発射 型 巡航 ミサイル を 実戦 配備 し た と 報じ た 


[[[cluster 10]]]
0) :金 正男 氏 殺害 で 女 1 人 を 逮捕 遺体 安置 の 病院 に は 北朝鮮 大使館 員 の 姿 も 北朝鮮 の 金 正 恩 （ キム ・ ジョン ウン 、 Kim Jong - Un ） 朝鮮 労働党 委員 長 の 異母 兄 、 金 正男 （ キム ・ ジョンナム 、 Kim Jong - Nam ） 氏 が マレーシア ・ クアラルンプール （ Kuala Lumpur ） の 空港 で 殺害 さ れ た 事件 で 、 マレーシア の 警察 当局 は 15 日 、 容疑 者 の 女 1 人 を 逮捕 し た 

1) :韓国 政府 、 金 正男 氏 の 暗殺 を 確認 大統領 代行 「 北 の 残忍 性 示す 」 （ 更新 ・ 写真 追加 ） 韓国 政府 は 15 日 、 北朝鮮 の 金 正 恩 （ キム ・ ジョン ウン 、 Kim Jong - Un ） 朝鮮 労働党 委員 長 の 異母 兄 、 金 正男 （ キム ・ ジョンナム 、 Kim Jong - Nam ） 氏 が マレーシア ・ クアラルンプール （ Kuala Lumpur ） の 空港 で 暗殺 さ れ た と の 報道 は 事実 だ と 確認 し た 

2) :北朝鮮 の 金 正男 氏 、 マレーシア で 暗殺 か 韓国 メディア 報道 （ 更新 、 写真 追加 ） 北朝

In [22]:
import sys
sys.path.append("../module/")
import align

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [117]:
scores = align.execute(base_sentence=targets_afp['title'].iloc[clusters[10][0]], target_sentences=targets['title'].tolist(), tokenize=tagger.parse, stps=stps, threshold=0.5).tolist()
scores = np.array(scores)

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/22c95b8e6878e472b6647f65afb10d88cafc52e0/variables/variables' with embeddings


In [120]:
targets['title'][scores > 1500]

10989    金正男氏殺害で女1人を逮捕 遺体安置の病院には北朝鮮大使館員の姿も
36523           金正男氏は北朝鮮の女性工作員2人が殺害＝韓国情報当局
36545            金正男氏の死亡、米は北朝鮮が殺害との見方＝米政府筋
Name: title, dtype: object

In [111]:
similarities = cosine_similarity(count_vectors, dense_output=False)
similarities2 = cosine_similarity(count_vectors, dense_output=True)

print("Mega bytes")
print(similarities.data.nbytes*1e-6, similarities2.data.nbytes*1e-6)
print("rate")
print(similarities2.data.nbytes / similarities.data.nbytes)

Mega bytes
0.946368 2.6912
rate
2.8437140731723813


In [181]:
similarities.data.nbytes

946368